---
# Demo - Safety and Security (Video Surveillance)

Live video feeds from 4 cameras are continuously ingested into SDP.

### Benefits of using SDP vs. a NAS-only solution

1. SDP provides low end-to-end latency (15-20 milliseconds) between a camera writing a frame and a video player displaying it.
   This is less than 1 frame at 60 frames per second.
   This low latency is impossible to achieve with file-only workflows due to limitations of locking, polling, NFS, and SMB.
2. There are strong guarantees of durability. Before the video player can display a frame, SDP writes it to SSDs on multiple
   tier 1 storage servers. In the event of system failure, data from tier 1 is used for recovery.
3. Data is *asynchronously* written to tier 2 (ECS or isilon) in large blocks, which means that cost-effective magnetic media can be used.
4. Older frames of video streams can be automatically truncated based on time or size limits.
5. The same API can be used to read live video or historical data from years ago.
   SDP will serve data cached in memory if available or it will read from tier 2.

---

### How to use this notebook
1. [Install dependencies](install_dependencies.ipynb).
2. [Begin ingesting data](ingest_kitti.ipynb).
3. [Run continuious index builder job](index_builder.ipynb).
2. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Initialize

In [ ]:
%load_ext autoreload
%autoreload 2

import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, VideoPlayer
from matplotlib import pyplot as plt
import IPython
import os
import ipywidgets
from ipywidgets import Layout
import numpy as np
import pandas as pd
import time
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets

imp.reload(video);

### Define Pravega stream parameters

#gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
#gateway = '10.246.27.131:54672'
gateway = 'pravega-grpc-gateway.examples.frightful-four.eaglemonk.intranet.nautilus-platform-dev.com:80'
scope = 'examples'
#stream = 'object-detector-input-video'
#stream = 'object-detector-output-video'
#stream = 'generated-video'
#stream = 'object-detector-input-grid-video'
#stream = 'virat'

### Initialize connection to Pravega GRPC Gateway

pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

---
### Video Player

In [ ]:
player = VideoPlayer(pravega_client=pravega_client, scope=scope)
player.interact()